In [20]:
import pandas as pd
import requests
import datetime

In [38]:
# Get data for all of 2023
start_date_time = "202301010000"
end_date_time = "202308250000"

# start_date_time = "202308250000"
# end_date_time =   "202309050000"

url = f"https://hourlypricing.comed.com/api?type=5minutefeed&datestart={start_date_time}&dateend={end_date_time}"

In [39]:
response = requests.get(url)

In [40]:
prices = response.json()

In [41]:
# Convert to a pandas dataframe
prices_df = pd.DataFrame(prices, dtype='float')

In [42]:
prices_df['millisUTC'] = prices_df['millisUTC'].apply(
    lambda x: datetime.datetime.fromtimestamp(
        float(x) / 1000.0, tz=datetime.timezone.utc)
)

prices_df['millisUTC'] = pd.to_datetime(prices_df['millisUTC'])

In [43]:
prices_df = prices_df.iloc[::-1].reset_index(drop=True)

In [44]:
def clean_up(data, resample_units=None):
     # reset_index
     # Convert the 'timestamp' column to datetime format and set it as the index
    data['millisUTC'] = pd.to_datetime(data['millisUTC'])
    data.set_index('millisUTC', inplace=True)

    # Resample dataset
    if resample_units is not None:
        data = data.resample(resample_units, label="right").mean()

    # Need a better way to handle missing values
    data['price'] = data['price'].ffill()
    data.reset_index(drop=True, inplace=True)
    return data

In [45]:
clean_up(prices_df)

,price
0,2.1
1,2.1
2,2.0
3,2.1
4,2.0
...,...
67121,2.2
67122,2.1
67123,2.2
67124,2.3


In [46]:
prices_df.to_csv("/Users/ankushgarg/Desktop/projects/comed-pricing/data/raw_data.csv", index=None)